# Программирование для всех<br>(основы работы с Python)

*Алла Тамбовцева*

## Практикум 7. Формат JSON и веб-страницы
### Краткое знакомство с форматом JSON

**JSON** (от *JavaScript Object Notation*) – текстовый формат хранения данных, изначально использовался в языке JavaScript, но затем стал универсальным машиночитаемым форматом, распознаваемым разными языками программирования. Различают:

* JSON-строки (текст с определённой структурой данных внутри);
* JSON-файлы (текстовые файлы с расширением `.json` со строкой JSON-внутри).

Какие структуры данных Python могут встретиться внутри JSON-строки? Знакомые нам списки и словари!

**Пример JSON-строки, содержащей списки:**

In [ ]:
# фрагмент результатов голосования в Арбитражный комитет Википедии: 
# время голосования, голос, кандидат, избиратель:

example01 = """
[["2008-11-23 00:32:00", "-", "Solon", "Kalan"], 
  ["2008-11-23 00:34:00", "+", "Chronicler", "Altes"], 
  ["2008-11-23 00:34:00", "+", "Ilya Voyager", "Altes"]]
"""

**Пример JSON-строки, содержащей словари:**

In [ ]:
# фрагмент результатов голосования за актеров 
# на сайте kinoteatr.ru

example02 = """
[{"id":"16804", "plus":131, "minus":4, "voted":""},
{"id":"56008", "plus":91, "minus":10, "voted":""},
{"id":"62460", "plus":94, "minus":4, "voted":""}]
"""

Этот формат хранения данных удобен своей универсальностью. Во-первых, он позволяет сохранять и выгружать в компактные текстовые файлы данные со сложной структурой (например, словари, внутри которых есть ещё словари). Во-вторых, формат JSON не привязан к какому-то конкретному языку программирования. Можно создать список списков в Python, выгрузить его в строку JSON, затем считать эту строку с помощью другого языка и получить результат в виде аналогичных структур данных, принятых в этом языке (например, аналогом питоновского словаря `dict` в языке R может выступить поименованный вектор или фрейм `list`).

По этим причинам формат JSON очень популярен. Его можно встретить при работе с географическими данными (файлы с особым расширением `.geojson`, которые содержат метки с координатами объектов), при парсинге HTML-страниц (файлы `.json`, из которых «подтягивается» регулярно обновляемая информация для построения всяких интерактивных визуализаций на сайте) и при подключении к API – интерфейсам, которые можно использовать как базы данных для автоматизированной выгрузки данных из приложений и социальных сетей.

В этом практикуме мы поработаем со строками JSON, используя разные модули и библиотеки.

### Чтение JSON-строк с модулем `json`

In [ ]:
import json

В модуле `json` есть две функции, `loads()` и `load()`. Первая преобразует данные из обычной строки (как здесь в примерах выше), вторая – данные, загруженные из файла (будет далее). Преобразование JSON-строки в структуру данных называется **десериализацией JSON**. 

Так, мы можем преобразовать строку в `example01` в список списков:

In [ ]:
res01 = json.loads(example01)
print(res01)

In [ ]:
# тип list
print(type(res01))

# можем пройти по нему в цикле и разобрать на части

timestamps = []
votes = []
candidates = []
voters = []

for r in res01:
    timestamps.append(r[0])
    votes.append(r[1])
    candidates.append(r[2])
    voters.append(r[3])
    
print(timestamps)
print(votes)
print(candidates)
print(voters)

А строку в `example02` – в список словарей:

In [ ]:
res02 = json.loads(example02)
print(res02)

> Вычислите «рейтинг» первого актера (id = 16804), считая, что рейтинг вычисляется как число лайков минус число дизлайков.

In [ ]:
### YOUR CODE HERE ###

Обратная операция – превращение структуры данных в Python в JSON-строку – тоже существует, и называется она **сериализацией JSON**. Для сериализации используется аналогичная пара функций, `dumps()` и `dump()`. Первая будет превращать структуру данных в JSON-строку, вторая – превращать структуру данных в строку и выгружать эту строку в файл с расширением `.json`.

Превращаем список в валидную JSON-строку:

In [ ]:
L = [{"user_id" : 1, "user_name" : "Anna", "last_seen" : "11-11-25 12:38:03"}, 
     {"user_id" : 2, "user_name" : "Dmitry", "last_seen" : "13-11-25 00:08:25"}]
my_json = json.dumps(L)
print(type(my_json))
print(my_json)

Примеры работы с JSON-файлами (и соответственно, функциями `load()` и `dump()`) – будут добавлены в конце файла с решениями практикума.

### Чтение JSON-строк из результатов запросов с модулем `requests`

Очень часто данные, которые используются для построения интерактивных таблиц и графиков на сайтах, хранятся в виде JSON-строки прямо внутри кода HTML или JSON-файла, ссылку на который тоже можно найти в коде HTML. Импортируем модуль `requests` для запросов:

In [ ]:
import requests

Посмотрим на пример данных в виде JSON-строки на странице с оценками фильма пользователей сайта Кинопоиск. Сначала познакомимся с самой страницей:

* откроем [страницу](https://www.kinopoisk.ru/film/44587/?utm_referrer=notebooks.githubusercontent.com) фильма «Господин оформитель»;
* заметим, что с нее можно перейти на [страницу](https://www.kinopoisk.ru/film/44587/votes/) с рейтингом и оценками;
* откроем исходный код страницы и найдем через поиск (`.json`) по странице ссылку на JSON-файл, откуда подгружаются данные для построения кольцевой диаграммы.

In [ ]:
url = "https://kinopoisk-fvs.s3.yandex.net/films/44587/value-stats.json"
print(url)

Отправим запрос к этой странице:

In [ ]:
page = requests.get(url, verify = False)
page

При работе с HTML-файлами мы просто запрашивали строку с исходным кодом страницы через атрибут `.text`, здесь это тоже возможно:

In [ ]:
print(page.text)

Как превратить такую строку в список словарей с модулем `json`, мы уже знаем, однако можно поступить проще. У объекта типа `Response` есть метод `.json()`, который забирает текст исходного кода страницы и, если в нем не код HTML, а JSON-строка, десериализует ее – преобразует в список словарей или список списков, в зависимости от ее структуры:

In [ ]:
for_chart = page.json()
for_chart

> Извлеките из `for_chart` оценку 10 и число таких оценок 1832.

In [ ]:
### YOUR CODE HERE ###

### Чтение JSON-строк с библиотекой  `pandas`

При работе с данными внутри JSON-строк по типу тех, которые мы уже рассмотрели, может возникнуть логичный вопрос: а нельзя ли из JSON-строки получить сразу таблицу, чтобы не разбирать полученный результат на части и не пересобирать из него что-то более удобное?

Конечно, можно, и для большого объема данных такой подход будет эффективнее. В исходном коде той же страницы есть ссылка на JSON-файл с данными по последним оценкам, поставленных фильму:

In [ ]:
relative = "/charts/votes/44587/last_votes.json"
print(relative)

Ссылка относительная, у нее нет «главной» части с `https` и названием сайта, чтобы она стала рабочей (и заодно кликабельной в Jupyter), добавим недостающую часть со ссылкой на сам сайт:

In [ ]:
main = "https://www.kinopoisk.ru"
relative = "/charts/votes/44587/last_votes.json"
link = main + relative
print(link)

Отправим запрос к этой странице, но заберем результат в виде обычного текста:

In [ ]:
req = requests.get(link, verify = False)
text_json = req.text

Импортируем `pandas` и подаем на вход функции с говорящим названием `read_json()` строку `text_json`:

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_json(text_json)
data

Готово! Можем работать с данными как с таблицей, что сильно удобнее, или выгрузить их в файл CSV или Excel для обработки в другой среде (Excel, RStudio и т д).